In [1]:
#Link checking mit csv-Datei:

#Importieren aller notwendigen Libraries:
import csv
import pandas as pd
import requests
from email.message import EmailMessage
import smtplib
from datetime import datetime

In [10]:
#Zusammenfügen von Report erstellen und E-Mail senden:
#Als erstes erstelle ich eine Funktion um eine E-Mail mit csv-Anhang an die Ziel-E-Mail-Addresse zu senden:

def send_email_alert(sender, recipient, report):
    from email.message import EmailMessage
    import smtplib
    
#header

    msg = EmailMessage()
    msg["Subject"] = "Broken Links in Reading List"
    #sender ='Theodor.sonnenschein@web.de' 
    msg['From'] = sender
    #recipient = 'Theodor.sonnenschein@web.de'
    msg['To'] = recipient

#Nachricht und Anhang
    date = datetime.now().strftime('%Y-%m-%d')
    
    msg.set_content("Dear Team, \nsome of the links are not working. Please have a look!")

    with open("broken_links_report.csv", "rb") as fp:
        msg.add_attachment(
            fp.read(), maintype= date + "_broken_links_report.csv", subtype="csv")
    
#Verbindung

    with smtplib.SMTP('smtp.web.de', 587) as smtplib:
        smtplib.ehlo()
        smtplib.starttls()
        smtplib.login(sender, 'password')
        smtplib.send_message(msg)

In [11]:
#Als nächstes schreibe ich eine Funktion für eine E-Mail im Falle, dass alle Links in Ordnung sind. 

def all_ok(sender, recipient):

    from email.message import EmailMessage
    import smtplib

#Header
    msg = EmailMessage()
    msg["Subject"] = "All links are ok - EOM"
    sender ='Theodor.sonnenschein@web.de' 
    msg['From'] = sender
    recipient = 'Theodor.sonnenschein@web.de'
    msg['To'] = recipient

#Nachricht
    msg.set_content(" ")


# Verbindung
    with smtplib.SMTP('smtp.web.de', 587) as s:
        s.ehlo()
        s.starttls()
        s.login(sender, 'password')
        s.send_message(msg)

In [6]:
#Anfang des eigentlichen Programms. Einlesen der csv-Datei mit Pandas, die vorher am bestimmten Ort abgespeichert wurde:

reading_list = pd.read_csv("Criminal_justice_readinglist.csv")

In [7]:
#isolieren der einzulesenden Spalte mit den URLs, wobei alle Einträge rausgelassen werden, die keine Onlineressourcen sind:
links = reading_list["url"].dropna()
links


0      http://www.centreformentalhealth.org.uk/mh-cri...
1      https://www.gov.uk/government/publications/sup...
2      http://www.cwp.nhs.uk/news-and-blogs/news/auti...
3      https://www.gov.uk/government/news/new-advice-...
4      https://www.lancashirecare.nhs.uk/blog/Crimina...
                             ...                        
193    http://tcr.sagepub.com/cgi/content/abstract/9/...
195    http://ehq.sagepub.com/cgi/content/abstract/35...
196    http://cmc.sagepub.com/cgi/content/abstract/2/...
198    http://www.ijtsrd.com/other-scientific-researc...
199    http://youthandpolicy.org/index.php?option=com...
Name: url, Length: 185, dtype: object

In [38]:
#Das Programm soll die URLs in der angegeben Spalte mit dem Modul "request" überprüfen und Ergebnisse in einer neuen Liste speichern:

broken_links = []

for url in links:
    try:
        r = requests.head(url)
        r.raise_for_status()
        
    #except requests.exceptions.InvalidURL:
            #pass
    #except requests.exceptions.MissingSchema:
            #pass
    # All Ausnahmen werden in einer Liste abgespeichert:
    except requests.exceptions.HTTPError as http_error:
        
           broken_links.append(f"{url}; {http_error}")
            
    except requests.exceptions.RequestException as other_error:
            broken_links.append(f"{url}; {other_error}")
            
    

#Nach Beenden des Überprüfens sollen die Ergebnisse in einer Tabelle gespeichert und per E-mail verschickt werden:
#Falls alle Links in Ordnung sind, wird eine einfache Info-E-mail verschickt:

if len(broken_links) == 0:
    all_ok('Theodor.sonnenschein@web.de', 'Theodor.sonnenschein@web.de')

#Gibt es Fehlermeldungen, wird die erstellte Tabelle in eine csv-Datei umgewandelt und 
#per E-Mail verschickt:

Create_report(broken_links)

send_email_alert('Theodor.sonnenschein@web.de', 'Theodor.sonnenschein@web.de', report)

In [36]:
def Create_report(report):
    
    df = pd.DataFrame(broken_links)           
    report = df[0].str.split(';', expand=True)            
    report.columns=["Item Links", "Error Status"]
    report.to_csv("broken_links_report.csv", index = False)

In [39]:
data = pd.read_csv("broken_links_report.csv")

In [40]:
data

,Item Links,Error Status
0,https://www.lancashirecare.nhs.uk/blog/Crimina...,HTTPSConnectionPool(host='www.lancashirecare....
1,http://www.parliament.uk/business/committees/c...,403 Client Error: Forbidden for url: http://w...
2,http://www.lanner.com/en/industries/industry.c...,404 Client Error: Not Found for url: http://w...
3,http://www.parliament.uk/business/committees/c...,403 Client Error: Forbidden for url: http://w...
4,https://onlinelibrary.wiley.com/doi/abs/10.100...,503 Server Error: Service Temporarily Unavail...
5,https://onlinelibrary.wiley.com/doi/abs/10.111...,503 Server Error: Service Temporarily Unavail...
6,https://onlinelibrary.wiley.com/doi/abs/10.100...,503 Server Error: Service Temporarily Unavail...
7,https://onlinelibrary.wiley.com/doi/abs/10.100...,503 Server Error: Service Temporarily Unavail...
8,http://www.parliament.uk/business/committees/c...,403 Client Error: Forbidden for url: http://w...
9,https://www.parliament.uk/business/committees/...,403 Client Error: Forbidden for url: https://...
